# Data-driven logic with Boolean data in Dyalog APL
Here are some basic Boolean array techniques followed by two practical examples of their use.

APL in general does not have typed data. Boolean data is not different. It is just like any other numeric data. It is just found to be Boolean if the values are only 0 or 1. Boolean data usually comes from comparisons like: 

In [1]:
'abcde'∊'be'

0 1 0 0 1

or

In [2]:
5>3 4 5 6 7

1 1 0 0 0

It can, of course, be used in Boolean logical operations:

In [3]:
1 0 1 ∧ 1 1 0   ⍝ Logical AND

1 0 0

In [4]:
1 0 1 ∨ 1 1 0   ⍝ Logical OR

1 1 1

But then it is also useful for counting occurances of a true statement:

In [5]:
+⌿'happy array programming'∊'apl'   ⍝ How many occurances of a, p or l?

7

or selecting pieces of an array corrensponding to the 1s:

In [6]:
bool ← 0 0 1 1 0 0 1 0 1
↑bool'ABCDEFGHI'

0 0 1 1 0 0 1 0 1
A B C D E F G H I

In [7]:
bool⌿'ABCDEFGHI'   ⍝ Select letters corresponding to 1s in bool

CDGI

For a 2-dimensional array, or **matrix**, a Boolean vector (list) can be used to select **rows** of the matrix:

In [8]:
⎕ ← names ← 9 6⍴'Adam  BertilCesar David Erik  Filip GustavHelge Ivar  '

Adam 
Bertil
Cesar 
David 
Erik 
Filip 
Gustav
Helge 
Ivar

In [9]:
bool⌿names

Cesar 
David 
Gustav
Ivar

Indices, or locations of `1`s in the vector can be easily found:

In [10]:
bool⌿⍳≢bool

3 4 7 9

This pattern is so common that it is the monadic form of *iota-underbar*, called **where**:

In [11]:
⍸bool

3 4 7 9

Now, before we take this a bit further, a quick note on implementation.

Booleans in Dyalog APL are implemented as 8 bits per byte. As explained in Marshall Lochbaum's 2017 talk on [Moving Bits Faster in Dyalog 16.0](https://dyalog.tv/Dyalog17/?v=2KnrDmZov4U), bit-Booleans only really benefit arrays and are difficult to implement, so most languages use a whole byte for their Boolean data type. However, Dyalog gets a huge performance boost for certain operations - especially on modern hardware where memory bandwidth (the time taken to move data around) is the main limiting factor.

## Production capacity per product per week

For this task, we want to estimate the number of products our organisation produces. We produce 7 types of products, labelled A, B, C, D, E, F and G.

But we have capacity constraints - we cannot produce however many products we need whenever we need them.

A capacity may be a production capacity, a logistic capacity or a market capacity.

- Production capacity is estimated from things like how quickly a machine can produce something, or how many hours somebody can work in a day.
- Logistic capacity might be how often and how much is the supply of parts.
- A market capacity is some kind of restriction on delivery of products - a market quota or transportation capacity.

Capacities are expressed as *remaining capacity per week*. In our organisation we have capacities labelled 1 to 5.

|Capacity\Week|2021w46|2021w47|2021w48|2021w49|2021w50|2021w51|2021w52|
|---|---|---|---|---|---|---|---|
| **1**  | 0  | 100  | 150  | 150  | 150  | 150  | 150 |
| **2**  | 0  | 150  | 200  | 200  | 300  | 300  | 300 |
| **3**  | 5  | 0    | 30   | 40   | 90   | 90   | 90  |
| **4**  | 0  | 50   | 0    | 60   | 30   | 0    | 45  |
| **5**  | 0  | 30   | 40   | 30   | 0    | 101  | 99  |

Let us define these availabile capacities as a numeric matrix called `avail`.

Although here we use a long APL expression to define `avail`, in practise this kind of data might be imported from somewhere like a spreadsheet application such as Microsoft Excel. 

In [12]:
⎕ ← avail ← 5 7⍴0 100,(5/150),0 150 200 200,(3/300),5 0 30 40,(3/90),0 50 0 60 30 0 45 0 30 40 30 0 101 99

0 100 150 150 150 150 150
0 150 200 200 300 300 300
5 0 30 40 90 90 90
0 50 0 60 30 0 45
0 30 40 30 0 101 99

We also know which capacities are utilised for supplying, manufacturing and delivering each product.

Product A uses capacities 1 and 2.  
Product B uses capacities 2 and 4.  
Product C uses capacity 1.  
etc.

These relationships can be mapped out in a table:

|| 1  | 2  | 3  | 4  | 5 |
|---|---|---|---|---|---|
| A  |  1  |  1  |  0  |  0  |  0 |
| B  |  0  |  1  |  0  |  1  |  0 |
| C  |  1  |  0  |  0  |  0  |  0 |
| D  |  0  |  0  |  1  |  0  |  1 |
| E  |  0  |  1  |  0  |  0  |  1 |
| F  |  0  |  0  |  1  |  1  |  0 |
| G  |  0  |  1  |  1  |  1  |  0 |

Let's call this the product-capacity cross matrix:

In [13]:
⎕ ← cross ← 7 5⍴1 1,(4/0),1 0 1 0 1,(6/0),1 0 1 0 1 0 0 1 0 0 1 1 0 0,(3/1),0

1 1 0 0 0
0 1 0 1 0
1 0 0 0 0
0 0 1 0 1
0 1 0 0 1
0 0 1 1 0
0 1 1 1 0

We will now investigate two solutions to finding the available production capacities for all five products A-G, across each of our seven weeks.

For clarity, here is the expected result:

|Product\Week|2021w46|2021w47|2021w48|2021w49|2021w50|2021w51|2021w52|
|---|---|---|---|---|---|---|---|
|**A**|0  |100  |150  |150  |150  |150  |150 |
|**B**|0  |50   |0    |60   |30   |0    |45  |
|**C**|0  |100  |150  |150  |150  |150  |150 |
|**D**|0  |0    |30   |30   |0    |90   |90  |
|**E**|0  |30   |40   |30   |0    |101  |99  |
|**F**|0  |0    |0    |40   |30   |0    |45  |
|**G**|0  |0    |0    |40   |30   |0    |45  |


We can see that, for example, the first available slot to produce **product G** is **2021w49**, and in that week we expect to produce **40 units**.

Now, here are the two APL solutions:

In [14]:
↑⌊⌿¨(↓cross)⌿¨⊂avail   ⍝ Solution 1

0 100 150 150 150 150 150
0 50 0 60 30 0 45
0 100 150 150 150 150 150
0 0 30 30 0 90 90
0 30 40 30 0 101 99
0 0 0 40 30 0 45
0 0 0 40 30 0 45

In [15]:
cross(⌊⌿⍤⌿⍤1 2)avail   ⍝ Solution 2

0 100 150 150 150 150 150
0 50 0 60 30 0 45
0 100 150 150 150 150 150
0 0 30 30 0 90 90
0 30 40 30 0 101 99
0 0 0 40 30 0 45
0 0 0 40 30 0 45

### Solution 1
`↑⌊⌿¨(↓cross)⌿¨⊂avail`

Both solutions do the same two essential computations:
1. Select rows from `avail` corresponding to each product, as provided in `cross`
2. Find the lowest availability in each week from those selected

Solution 1 does this as two data transformation steps and uses the [**each**](https://mastering.dyalog.com/Nested-Arrays-Continued.html#each) operator (`F¨`) to apply functions for each product.

First, split the `cross` matrix into a [nested vector of Boolean vectors](https://mastering.dyalog.com/Data-and-Variables.html#mixed-and-nested-arrays):

In [16]:
↓cross

┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│1 1 0 0 0│0 1 0 1 0│1 0 0 0 0│0 0 1 0 1│0 1 0 0 1│0 0 1 1 0│0 1 1 1 0│
└─────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

In order to compress rows from *the whole of* `avail` using each Boolean vector in `cross`, we must [enclose](https://mastering.dyalog.com/Nested-Arrays-Continued.html?highlight=enclose#enclose-disclose) (`⊂⍵`) it. 

<details>
    <summary>
        Why does <code>avail</code> need enclosing? [click for more information]</summary>
    </summary>
    The Each <code>F¨</code> operator either loops through two arrays of the same shape, applying its operand function <code>F</code> to corresponding elements in each array, or <em>if one of the arrays is a scalar (rank-0 array)</em>, then it loops over the non-scalar array, applying between the scalar argument and each element in the non-scalar argument. 
    <br><br>
In this case, we want to apply the <strong>compress</strong> function <code>⍺⌿⍵</code> between each Boolean vector in <code>cross</code> and the whole table. We use the <strong>enclose</strong> function <code>⊂⍵</code> to wrap the <code>avail</code> matrix as a nested scalar. `⊂avail` (<em>enclose avail</em>) is a nested scalar which contains a numeric matrix.
    <br><br>
Another example of this same idea is shown in <a target="_blank" rel="noopener noreferrer" href="https://mastering.dyalog.com/Nested-Arrays-Continued.html#scalar-dyadic-functions">section 10.5.1 of Mastering Dyalog APL</a>.
</details>

In [17]:
(↓cross)⌿¨⊂avail

┌─────────────────────────┬─────────────────────────┬─────────────────────────┬────────────────────┬─────────────────────────┬───────────────────┬─────────────────────────┐
│0 100 150 150 150 150 150│0 150 200 200 300 300 300│0 100 150 150 150 150 150│5 0 30 40 90 90 90│0 150 200 200 300 300 300│5 0 30 40 90 90 90│0 150 200 200 300 300 300│
│0 150 200 200 300 300 300│0 50 0 60 30 0 45│ │0 30 40 30 0 101 99│0 30 40 30 0 101 99│0 50 0 60 30 0 45│5 0 30 40 90 90 90│
│ │ │ │ │ │ │0 50 0 60 30 0 45│
└─────────────────────────┴─────────────────────────┴─────────────────────────┴────────────────────┴─────────────────────────┴───────────────────┴─────────────────────────┘

So now we have the available capacities required for each product:

In [18]:
prod ← 'ABCDEFG'
prod,⍪(↓cross)⌿¨⊂avail

┌─┬─────────────────────────┐
│A│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
├─┼─────────────────────────┤
│B│0 150 200 200 300 300 300│
│ │0 50 0 60 30 0 45│
├─┼─────────────────────────┤
│C│0 100 150 150 150 150 150│
├─┼─────────────────────────┤
│D│5 0 30 40 90 90 90 │
│ │0 30 40 30 0 101 99 │
├─┼─────────────────────────┤
│E│0 150 200 200 300 300 300│
│ │0 30 40 30 0 101 99│
├─┼─────────────────────────┤
│F│5 0 30 40 90 90 90 │
│ │0 50 0 60 30 0 45 │
├─┼─────────────────────────┤
│G│0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
└─┴─────────────────────────┘

We find the least of these for each week (that is, column-wise):

In [19]:
⌊⌿¨(↓cross)⌿¨⊂avail

┌─────────────────────────┬─────────────────┬─────────────────────────┬─────────────────┬───────────────────┬────────────────┬────────────────┐
│0 100 150 150 150 150 150│0 50 0 60 30 0 45│0 100 150 150 150 150 150│0 0 30 30 0 90 90│0 30 40 30 0 101 99│0 0 0 40 30 0 45│0 0 0 40 30 0 45│
└─────────────────────────┴─────────────────┴─────────────────────────┴─────────────────┴───────────────────┴────────────────┴────────────────┘

And finally [**Mix**](https://mastering.dyalog.com/Nested-Arrays-Continued.html#split-and-mix) this nested vector into a matrix. We prefer a simple matrix over a nested vector because it takes less memory to store and is generally easier to use for further calculations.

### Solution 2
`cross(⌊⌿⍤⌿⍤1 2)avail`

As a reminder, here are the two essential computations:
1. Select rows from `avail` corresponding to each product, as provided in `cross`
2. Find the lowest availability in each week from those selected

Solution 2 uses a function composition operator, called [**atop**]() (`F⍤G`) to package these two steps into a single function. Our combined function applys `⍺⌿⍵` between our two arguments, and `⌊⌿⍵` to the result. For a single row of `cross`, this work as-is.

In [20]:
cross[2;] ⌊⌿⍤⌿ avail   ⍝ Minimum reduction atop compress

0 50 0 60 30 0 45

However, we want to apply this function between each **row** (rank-1 array) of `cross` and the **whole** (rank-2 array) of `avail`. The [**rank operator**](https://xpqz.github.io/learnapl/rank.html) lets us put "[blinders](https://en.wikipedia.org/wiki/Blinkers_(horse_tack))" on our function so that it can only see parts of our arrays that we tell it to.

Let's start with a smaller example. Here `{⍺⍵}` is a function which returns its arguments:

In [21]:
1 2 3 {⍺⍵} 3 2⍴'ABCD'

┌─────┬──┐
│1 2 3│AB│
│ │CD│
│ │AB│
└─────┴──┘

Let's use this function `{⍺⍵}` to pair up each number (rank 0 scalar) of `⍺` and each row (rank 1 vector) of `⍵`.

<details>
<summary>Why is <code>({⍺⍵}⍤0 1)</code> parenthesised? [click for more information]</summary>
Parentheses are required here to prevent our array right operand <code>0 1</code> from being confused with our right argument <code>3 2⍴⎕A</code>.
        
<pre><code>
      1 2 3 {⍺⍵}⍤0 1 3 2⍴⎕A
LENGTH ERROR
      1 2 3{⍺ ⍵}⍤0 1 3 2⍴⎕A
                ∧
</code></pre>
    
You might also see the right-tack `⊢` used for the same effect:
    
<pre><code>
1 2 3{⍺⍵}⍤0 1⊢3 2⍴⎕A
</code></pre>
    
</details>

In [22]:
1 2 3 ({⍺⍵}⍤0 1) 3 2⍴⎕A 

┌─┬──┐
│1│AB│
├─┼──┤
│2│CD│
├─┼──┤
│3│EF│
└─┴──┘

Or we can pair each scalar with each matrix (rank 2) of our right argument. Since our right argument *is* a matrix, the entire array is used repeatedly.

In [23]:
1 2 3 ({⍺⍵}⍤0 2) 3 2⍴⎕A

┌─┬──┐
│1│AB│
│ │CD│
│ │EF│
├─┼──┤
│2│AB│
│ │CD│
│ │EF│
├─┼──┤
│3│AB│
│ │CD│
│ │EF│
└─┴──┘

Now back to our manufacturing problem. We want rows (rank 1) of `cross` used with the whole (rank 2) of `avail`.

In [24]:
cross ({⍺⍵}⍤1 2) avail

┌─────────┬─────────────────────────┐
│1 1 0 0 0│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
├─────────┼─────────────────────────┤
│0 1 0 1 0│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
├─────────┼─────────────────────────┤
│1 0 0 0 0│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
├─────────┼─────────────────────────┤
│0 0 1 0 1│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
├─────────┼─────────────────────────┤
│0 1 0 0 1│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
├─────────┼─────────────────────────┤
│0 0 1 1 0│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
├─────────┼─────────────────────────┤
│0 1 1 1 0│0 100 150 150 150 150 150│
│ │0 150 200 200 300 300 300│
│ │5 0 30 40 90 90 90│
│ │0 50 0 60 30 0 45│
│ │0 30 40 30 0 101 99│
└─────────┴─────────────────────────┘

Essentially, the rank operator controls what its operand function "sees" as its arguments. Conceptually we are looping over our arguments, although the implementation of some functions applied with rank could be parallel instead.

Now that our arguments are correctly paired up, we simply put our function `⌊⌿⍤⌿` in place of `{⍺⍵}` and we are done:

In [25]:
cross (⌊⌿⍤⌿⍤1 2) avail

0 100 150 150 150 150 150
0 50 0 60 30 0 45
0 100 150 150 150 150 150
0 0 30 30 0 90 90
0 30 40 30 0 101 99
0 0 0 40 30 0 45
0 0 0 40 30 0 45

## Useful array representations
Getting other interesting information from this data is also easy:

The number of capacities used per product:

In [26]:
+/cross   ⍝ This is a row-wise sum

2 2 1 2 2 2 3

The number of products which use each capacity:

In [27]:
+⌿cross

2 4 3 3 2

The list of products which use capacity 2:

In [28]:
cross[;2]⌿prod

ABEG

The use of Boolean values to select from arrays, as shown here, is one of the most common patterns in APL. Boolean representations are very space efficient, and APL implementations are usually optimised for operations on Boolean arrays.